In [1]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import imageio
import numpy as np
import warnings
import torch
from torch import nn
import torch.optim as opt

In [48]:
plays=pd.read_csv('plays.csv')
rush_plays=plays.loc[plays['passResult'].isna()==True]
rush_plays=rush_plays[['gameId','playId', 'passResult']]

rush_plays

,gameId,playId,passResult
1,2022091103,3126,NaN
3,2022100212,2007,NaN
4,2022091900,1372,NaN
7,2022102310,56,NaN
8,2022102310,1318,NaN
...,...,...,...
12477,2022102307,1224,NaN
12481,2022100204,123,NaN
12483,2022101605,3371,NaN
12484,2022100207,2777,NaN


In [379]:
df1=pd.read_csv('CounterFactualsweek_1.csv', index_col=None)
df2=pd.read_csv('CounterFactualsweek_2.csv', index_col=None)
df3=pd.read_csv('CounterFactualsweek_3.csv', index_col=None)
df4=pd.read_csv('CounterFactualsweek_4.csv', index_col=None)
df5=pd.read_csv('CounterFactualsweek_5.csv', index_col=None)
df6=pd.read_csv('CounterFactualsweek_6.csv', index_col=None)
df7=pd.read_csv('CounterFactualsweek_7.csv', index_col=None)
df8=pd.read_csv('CounterFactualsweek_8.csv', index_col=None)
df9=pd.read_csv('CounterFactualsweek_9.csv', index_col=None)

df_counterfactuals=pd.concat([df1,df2, df3, df4,df5, df6, df7,df8, df9])

df_counterfactuals.sort_values(by='Prediction Difference', ascending=False)

,Unnamed: 0,nflId,gameId,playId,frameId,Yards Remaining,Original Prediction,Projected Prediction,Prediction Difference
70216,70216,47882.0,2022102301,2441,53,46.09,6.678520,26.992758,20.314238
70215,70215,47882.0,2022102301,2441,52,46.84,7.474641,26.960474,19.485833
70217,70217,47882.0,2022102301,2441,54,45.33,5.951662,25.384060,19.432398
276323,276323,47846.0,2022110610,1419,42,44.45,7.037872,26.234985,19.197114
272440,272440,42488.0,2022103009,1244,72,2.63,6.648518,25.558067,18.909549
...,...,...,...,...,...,...,...,...,...
113215,113215,44872.0,2022091103,1126,29,23.13,14.272851,5.963061,-8.309790
113214,113214,44872.0,2022091103,1126,28,23.85,17.032185,8.651917,-8.380267
25994,25994,52792.0,2022091500,3780,9,27.90,16.375103,6.209756,-10.165347
243965,243965,53462.0,2022100209,740,25,4.00,NaN,NaN,NaN


In [85]:
players=pd.read_csv('players.csv')
players=players[['nflId', 'displayName', 'position']]


In [148]:
check_df=df.merge(players, how='left', on='nflId')
check_df=check_df.loc[((check_df['position']=='SS') | (check_df['position']=='FS') | (check_df['position']=='CB'))]
check_df['Prediction Difference'].mean()

0.03232538385886889

In [150]:
df_num_plays=pd.DataFrame(df.groupby(['nflId'])['playId'].nunique())
df_num_frames=pd.DataFrame(df['nflId'].value_counts())
df_vals=pd.DataFrame(df.groupby(['nflId'])['Prediction Difference'].mean()).merge(players, on='nflId').merge(df_num_plays, how='left', on='nflId').merge(df_num_frames, how='left', on='nflId')
df_vals['Prediction Difference']=(df_vals['Prediction Difference']-0.03232538385886889)*df_vals['count']
df_vals=df_vals.sort_values(by='Prediction Difference', ascending=False)
# df_vals=df_vals[((df_vals['position']=='DE') | (df_vals['position']=='DT') | (df_vals['position']=='NT')) & (df_vals['playId']>100)]
# df_vals=df_vals[((df_vals['position']=='OLB') | (df_vals['position']=='ILB') | (df_vals['position']=='MLB')) & (df_vals['playId']>100)]
df_vals=df_vals[((df_vals['position']=='SS') | (df_vals['position']=='FS') | (df_vals['position']=='CB')) & (df_vals['playId']>100)]
df_vals.head(50)
# df=df.groupby(['nflId', 'playId']).sum()
# df.loc[df['nflId']==42901]
# distinct_counts=df.groupby(['nflId'])['playId'].nunique()

,nflId,Prediction Difference,displayName,position,playId,count
435,47882.0,851.351100,Mike Edwards,FS,220,6545
595,52852.0,316.943547,Myles Hartsfield,FS,299,8629
216,44848.0,310.423232,Budda Baker,SS,314,9376
218,44854.0,293.426866,Marcus Williams,FS,164,4549
643,53503.0,275.240831,Benjamin St-Juste,CB,257,7919
154,43350.0,264.244370,Vonn Bell,SS,285,8336
406,47810.0,230.319103,Johnathan Abram,SS,198,5674
735,54502.0,200.575448,Jalen Pitre,FS,262,8822
247,44925.0,192.976617,Eddie Jackson,SS,289,9101
248,44926.0,188.911063,Rayshawn Jenkins,SS,272,7901


In [251]:
def get_tracking_with_plays(filepath):
    tracking = pd.read_csv(filepath)
    plays = pd.read_csv("plays.csv")
    tracking = tracking[tracking['playId'].isin(tracking[tracking['event'] != 'fumble']['playId'].unique())]
    plays = plays[plays['playNullifiedByPenalty'] == 'N']
    tracking.loc[tracking['playDirection'] == 'left', 'x'] = 120 - tracking.loc[tracking['playDirection'] == 'left', 'x']
    tracking.loc[tracking['playDirection'] == 'left', 'y'] = (160/3) - tracking.loc[tracking['playDirection'] == 'left', 'y']
    tracking.loc[tracking['playDirection'] == 'left', 'dir'] += 180
    tracking.loc[tracking['dir'] > 360, 'dir'] -= 360
    tracking.loc[tracking['playDirection'] == 'left', 'o'] += 180
    tracking.loc[tracking['o'] > 360, 'o'] -= 360
    tracking_with_plays = tracking.merge(plays, on=['gameId', 'playId'], how='left')
    tracking_with_plays['is_on_offense'] = tracking_with_plays['club'] == tracking_with_plays['possessionTeam']
    tracking_with_plays['is_on_defence'] = tracking_with_plays['club'] == tracking_with_plays['defensiveTeam']
    tracking_with_plays['is_ballcarrier'] = tracking_with_plays['ballCarrierId'] == tracking_with_plays['nflId']
    bc_coords=tracking_with_plays.loc[tracking_with_plays['is_ballcarrier']]
    bc_coords['bc_x']=bc_coords['x']
    bc_coords['bc_y']=bc_coords['y']
    bc_coords['prev_bc_x']= bc_coords.groupby(['gameId', 'playId'])['bc_x'].transform(lambda x: x.shift(1))
    bc_coords['prev_bc_y']= bc_coords.groupby(['gameId', 'playId'])['bc_y'].transform(lambda x: x.shift(1))
    bc_coords['change_in_x']=bc_coords['bc_x']-bc_coords['prev_bc_x']
    bc_coords=bc_coords[['gameId', 'playId', 'frameId', 'bc_x', 'bc_y', 'prev_bc_x', 'prev_bc_y', 'change_in_x']]
    tracking_with_plays=tracking_with_plays.merge(bc_coords, on=['gameId', 'playId', 'frameId'], how='left')
    end_frame = tracking_with_plays[tracking_with_plays['event'].isin(['tackle', 'out_of_bounds'])].groupby(['gameId', 'playId'])['frameId'].min().reset_index()
    end_frame.rename(columns={'frameId': 'frameId_end'}, inplace=True)
    start_frame = tracking_with_plays[tracking_with_plays['event'].isin(['run', 'lateral', 'run_pass_option', 'handoff', 'pass_arrived'])].groupby(['gameId', 'playId'])['frameId'].min().reset_index()
    start_frame.rename(columns={'frameId': 'frameId_start'}, inplace=True)
    tracking_with_plays = tracking_with_plays.merge(start_frame, on=['gameId', 'playId'], how='left')
    tracking_with_plays = tracking_with_plays.merge(end_frame, on=['gameId', 'playId'], how='left')
    tracking_with_plays = tracking_with_plays[(tracking_with_plays['frameId'] <= tracking_with_plays['frameId_end']) &
                                              (tracking_with_plays['frameId'] >= tracking_with_plays['frameId_start'])]
    tracking_with_plays['distance_from_bc']=(((tracking_with_plays['x']-tracking_with_plays['bc_x'])**2)+(tracking_with_plays['y']-tracking_with_plays['bc_y'])**2)**(1/2)
    return tracking_with_plays

plays = pd.read_csv('plays.csv')
games = pd.read_csv('games.csv')
tracking1 = get_tracking_with_plays('tracking_week_1.csv')
tracking2 = get_tracking_with_plays('tracking_week_2.csv')
tracking3 = get_tracking_with_plays('tracking_week_3.csv')
tracking4 = get_tracking_with_plays('tracking_week_4.csv')
tracking5 = get_tracking_with_plays('tracking_week_5.csv')
tracking6 = get_tracking_with_plays('tracking_week_6.csv')
tracking7 = get_tracking_with_plays('tracking_week_7.csv')
tracking8 = get_tracking_with_plays('tracking_week_8.csv')
tracking9 = get_tracking_with_plays('tracking_week_9.csv')

tracking_total=pd.concat([tracking1, tracking2, tracking3, tracking4, tracking5, tracking6, tracking7, tracking8, tracking9])
tracking_total

/var/folders/pq/qr32tkk90n7361gslj40w8j4tyz0cb/T/ipykernel_65795/2457732242.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bc_coords['bc_x']=bc_coords['x']
/var/folders/pq/qr32tkk90n7361gslj40w8j4tyz0cb/T/ipykernel_65795/2457732242.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bc_coords['bc_y']=bc_coords['y']
/var/folders/pq/qr32tkk90n7361gslj40w8j4tyz0cb/T/ipykernel_65795/2457732242.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

,gameId,playId,nflId,displayName,frameId,time,jerseyNumber,club,playDirection,x,...,is_on_defence,is_ballcarrier,bc_x,bc_y,prev_bc_x,prev_bc_y,change_in_x,frameId_start,frameId_end,distance_from_bc
1,2022090800,56,35472.0,Rodger Saffold,2,2022-09-08 20:24:05.299999,76.0,BUF,left,31.530000,...,False,False,39.52,15.653333,39.40,15.043333,0.12,2.0,18.0,13.234145
2,2022090800,56,35472.0,Rodger Saffold,3,2022-09-08 20:24:05.400000,76.0,BUF,left,31.440000,...,False,False,39.66,16.243333,39.52,15.653333,0.14,2.0,18.0,13.006721
3,2022090800,56,35472.0,Rodger Saffold,4,2022-09-08 20:24:05.500000,76.0,BUF,left,31.360000,...,False,False,39.81,16.803333,39.66,16.243333,0.15,2.0,18.0,12.811690
4,2022090800,56,35472.0,Rodger Saffold,5,2022-09-08 20:24:05.599999,76.0,BUF,left,31.280000,...,False,False,39.99,17.333333,39.81,16.803333,0.18,2.0,18.0,12.669021
5,2022090800,56,35472.0,Rodger Saffold,6,2022-09-08 20:24:05.700000,76.0,BUF,left,31.200000,...,False,False,40.15,17.743333,39.99,17.333333,0.16,2.0,18.0,12.614856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150014,2022110700,3787,NaN,football,36,2022-11-07 23:06:48.799999,NaN,football,right,25.540001,...,False,False,25.37,20.230000,25.05,20.400000,0.32,15.0,40.0,0.233453
1150015,2022110700,3787,NaN,football,37,2022-11-07 23:06:48.900000,NaN,football,right,25.760000,...,False,False,25.65,20.070000,25.37,20.230000,0.28,15.0,40.0,0.178045
1150016,2022110700,3787,NaN,football,38,2022-11-07 23:06:49.000000,NaN,football,right,25.950001,...,False,False,25.90,19.930000,25.65,20.070000,0.25,15.0,40.0,0.094340
1150017,2022110700,3787,NaN,football,39,2022-11-07 23:06:49.099999,NaN,football,right,26.100000,...,False,False,26.08,19.810000,25.90,19.930000,0.18,15.0,40.0,0.053852


In [223]:
def get_tracking(week_num):
    if week_num==1:
        return tracking1
    elif week_num==2:
        return tracking2
    elif week_num==3:
        return tracking3
    elif week_num==4:
        return tracking4
    elif week_num==5:
        return tracking5
    elif week_num==6:
        return tracking6
    elif week_num==7:
        return tracking7
    elif week_num==8:
        return tracking8
    else:
        return tracking9

In [380]:
xY=pd.read_csv('ExpectedYards_v1.csv')
xY['ExpectedYards']=xY['ExpectedYards']*1.5
# xY.loc[xY['xY']]

In [381]:
df_counterfactuals['Prediction Difference']=df_counterfactuals['Prediction Difference']*1.5
df_counterfactuals['Original Prediction']=df_counterfactuals['Original Prediction']*1.5
df_counterfactuals['Projected Prediction']=df_counterfactuals['Projected Prediction']*1.5

In [382]:
tracking1=tracking_total.merge(xY, how='left', on=['gameId', 'playId', 'frameId'])
tracking1=tracking1.merge(df_counterfactuals, how='left', on=['nflId', 'gameId', 'playId', 'frameId'])
tracking1=tracking1.loc[tracking1['is_on_defence']==True]
tracking1['PrevExpectedYards']=tracking1.groupby(['gameId', 'playId', 'nflId'])['ExpectedYards'].transform(lambda x: x.shift(1))
tracking1=tracking1.loc[tracking1['PrevExpectedYards'].isna()==False]
tracking1['YardsSaved']=tracking1['PrevExpectedYards']-tracking1['change_in_x']-tracking1['ExpectedYards']
tracking1['defenseCumulativeDistance']=tracking1.groupby(['gameId', 'playId', 'frameId'])['distance_from_bc'].transform('sum')
tracking1['Prediction_Difference_Sign']=tracking1['Prediction Difference']*tracking1['YardsSaved']
tracking1['Prediction_Difference_Adjusted']=tracking1['Prediction_Difference_Sign'].apply(lambda value: 0 if value<0 else value)
# tracking1['Player_Proportional_Credit']=np.exp(-10*(tracking1['distance_from_bc']/tracking1['defenseCumulativeDistance']))
tracking1['Total_Credit']=tracking1.groupby(['gameId', 'playId', 'frameId'])['Prediction_Difference_Adjusted'].transform('sum')
tracking1['Player_Yards_Saved']=(tracking1['Prediction_Difference_Adjusted']/tracking1['Total_Credit'])*tracking1['YardsSaved']
tracking1.loc[(tracking1['gameId']==2022090800) & (tracking1['playId']==80) & (tracking1['frameId']==7)]

,gameId,playId,nflId,displayName,frameId,time,jerseyNumber,club,playDirection,x,...,Original Prediction,Projected Prediction,Prediction Difference,PrevExpectedYards,YardsSaved,defenseCumulativeDistance,Prediction_Difference_Sign,Prediction_Difference_Adjusted,Total_Credit,Player_Yards_Saved
413,2022090800,80,38577.0,Bobby Wagner,7,2022-09-08 20:24:36.299999,45.0,LA,left,48.44,...,8.6078,8.609854,0.002054,8.818153,-0.199647,154.67448,-0.000410,0.000000,0.001968,-0.000000
434,2022090800,80,41239.0,Aaron Donald,7,2022-09-08 20:24:36.299999,99.0,LA,left,32.01,...,8.6078,8.605315,-0.002485,8.818153,-0.199647,154.67448,0.000496,0.000496,0.001968,-0.050314
497,2022090800,80,42816.0,Troy Hill,7,2022-09-08 20:24:36.299999,2.0,LA,left,56.07,...,8.6078,8.610070,0.002270,8.818153,-0.199647,154.67448,-0.000453,0.000000,0.001968,-0.000000
518,2022090800,80,43294.0,Jalen Ramsey,7,2022-09-08 20:24:36.299999,5.0,LA,left,47.22,...,8.6078,8.605132,-0.002667,8.818153,-0.199647,154.67448,0.000532,0.000532,0.001968,-0.054007
539,2022090800,80,43298.0,Leonard Floyd,7,2022-09-08 20:24:36.299999,54.0,LA,left,34.81,...,8.6078,8.630930,0.023131,8.818153,-0.199647,154.67448,-0.004618,0.000000,0.001968,-0.000000
560,2022090800,80,43335.0,A'Shawn Robinson,7,2022-09-08 20:24:36.299999,94.0,LA,left,38.38,...,8.6078,8.616260,0.008461,8.818153,-0.199647,154.67448,-0.001689,0.000000,0.001968,-0.000000
644,2022090800,80,47844.0,Taylor Rapp,7,2022-09-08 20:24:36.299999,24.0,LA,left,54.33,...,8.6078,8.608790,0.000991,8.818153,-0.199647,154.67448,-0.000198,0.000000,0.001968,-0.000000
707,2022090800,80,47917.0,Greg Gaines,7,2022-09-08 20:24:36.299999,91.0,LA,left,40.17,...,8.6078,10.984713,2.376914,8.818153,-0.199647,154.67448,-0.474543,0.000000,0.001968,-0.000000
728,2022090800,80,48026.0,Nick Scott,7,2022-09-08 20:24:36.299999,33.0,LA,left,60.39,...,8.6078,8.607368,-0.000432,8.818153,-0.199647,154.67448,0.000086,0.000086,0.001968,-0.008748
791,2022090800,80,52607.0,Jordan Fuller,7,2022-09-08 20:24:36.299999,4.0,LA,left,57.63,...,8.6078,8.603524,-0.004276,8.818153,-0.199647,154.67448,0.000854,0.000854,0.001968,-0.086579


In [383]:
# sum(tracking1['Player_Yards_Saved'])
players=pd.read_csv('players.csv')[['displayName', 'position']]
results=pd.DataFrame(tracking1.groupby(['displayName'])['Player_Yards_Saved'].sum()).sort_values(by='Player_Yards_Saved', ascending=False).merge(players, how='left', on=['displayName'])
sum(results['Player_Yards_Saved'])

-864.8084447272863

In [384]:
results.loc[(results['position']=='ILB') | (results['position']=='OLB') | (results['position']=='MLB')].head(20)

,displayName,Player_Yards_Saved,position
1,Jordan Hicks,80.101240,ILB
2,Pete Werner,74.307533,OLB
3,Bobby Okereke,73.505401,ILB
4,Zaire Franklin,70.177729,OLB
6,C.J. Mosley,67.266459,ILB
7,Brian Burns,66.189000,OLB
8,Drue Tranquill,65.077314,ILB
9,Patrick Queen,62.778968,ILB
10,Tremaine Edmunds,60.838987,ILB
12,Eric Kendricks,58.277098,ILB


In [387]:
results.loc[(results['position']=='FS') | (results['position']=='SS') | (results['position']=='CB')].head(20)

,displayName,Player_Yards_Saved,position
0,Jordan Whitehead,90.624861,SS
5,Nick Scott,67.769808,SS
13,Roger McCreary,57.144110,CB
16,Jonathan Owens,55.608132,FS
19,Kyle Dugger,53.402134,FS
20,Vonn Bell,52.895089,SS
22,Eddie Jackson,52.113715,SS
23,Grant Delpit,51.808869,SS
25,Rayshawn Jenkins,50.622886,SS
29,Jordan Poyer,48.859522,SS


In [386]:
results.loc[(results['position']=='DT') | (results['position']=='NT') | (results['position']=='DE')].head(20)

,displayName,Player_Yards_Saved,position
11,Sam Hubbard,59.005316,DE
17,Grover Stewart,55.588621,DT
21,Kwity Paye,52.722436,DE
40,Jeffery Simmons,41.704227,DT
45,John Franklin-Myers,39.147220,DE
46,Christian Wilkins,37.663603,DT
47,Justin Madubuike,37.586068,DT
50,Sheldon Rankins,36.636351,DT
54,Osa Odighizuwa,35.326172,DT
55,Lawrence Guy,35.129526,DT
